In [2]:
import torch
from torch.utils import data
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import pandas as pd
import numpy as np
import scipy
from tqdm import trange
from tqdm import tqdm
from datetime import datetime
import sys
import os
import seaborn as sns
from matplotlib import pyplot as plt
from joblib import Parallel, delayed, dump, load
from matplotlib import pyplot as plt
from sparse_vector.sparse_vector import SparseVector
from scipy.signal import convolve2d, convolve
import time
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

/home/dumerenkov/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/dumerenkov/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#ZDNA_kouzine = {}
#ZDNA = load(f'../data/mm9_zdna/sparse/curax_14h_UNI_mm9.pkl')
#for chrom in ZDNA:
#    ZDNA_kouzine[chrom] = np.zeros(ZDNA[chrom].shape, dtype = bool)
#    
#    
#with open("Kouzine_mm9.bed")as f:
#    for idx, line in enumerate(f):
#        if idx>0:
#            chrom, start, end = line.strip().split()
#            if chrom in ZDNA_kouzine:
#                ZDNA_kouzine[chrom][int(start):int(end)] = 1
#
#dump(ZDNA_kouzine, 'ZDNA_mm9_kouzine.pkl')

In [4]:
ASSEMBLY_d = {}
chroms_d = {}
all_features_d = {}
groups_d = {}
feature_names_d = {}
ZDNA_d = {}
black_list_d = {}
DNA_d = {}
DNA_features_d = {}

# MM9

In [5]:
ASSEMBLY = "curax_14h_UNI_mm9"
chroms = [f'chr{i}' for i in list(range(1, 20)) + ['X', 'Y']]
all_features = sorted([i[:-4] for i in os.listdir('../data/mm9_features/sparse/') if i.endswith('.pkl')])
groups = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
feature_names = [i for i in all_features if (i.split('_')[0] in groups)]
ZDNA = load(f'ZDNA_mm9_kouzine.pkl')
black_list = load(f'../data/mm9_zdna/sparse/blacklist_mm9.pkl')


In [6]:
DNA = {chrom:load(f'../data/mm9_dna/sparse/{chrom}.pkl') for chrom in tqdm(chroms)}

DNA_features = {feture: load(f'../data/mm9_features/sparse/{feture}.pkl')
                for feture in tqdm(feature_names)}

for feature in tqdm(DNA_features):
    if set(DNA_features[feature].keys()) != set(chroms):
        for chrom in chroms:
            if chrom not in DNA_features[feature]:
                DNA_features[feature][chrom] = SparseVector(len(DNA[chrom]))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 873/873 [00:00<00:00, 231367.84it/s]


In [7]:
mode = 'mm9'
ASSEMBLY_d[mode] = ASSEMBLY
chroms_d[mode] = chroms
all_features_d[mode] = all_features
groups_d[mode] = groups
feature_names_d[mode] = feature_names
ZDNA_d[mode] = ZDNA
black_list_d[mode] = black_list
DNA_d[mode] = DNA
DNA_features_d[mode] = DNA_features

# HG19

In [8]:
ASSEMBLY = "ZDNA_2016"
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y']]
all_features = sorted([i[:-4] for i in os.listdir('../data/hg19_features/sparse/') if i.endswith('.pkl')])
groups = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
feature_names = [i for i in all_features if (i.split('_')[0] in groups)]
ZDNA = load('ZDNA_hg19_kouzine.pkl')
black_list = load(f'../data/hg19_zdna/sparse/blacklist_hg19.pkl')


In [9]:
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'../data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"../data/hg19_dna/{file}") for file in files])


DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

DNA_features = {feture: load(f'../data/hg19_features/sparse/{feture}.pkl')
                for feture in tqdm(feature_names)}

for feature in tqdm(DNA_features):
    if set(DNA_features[feature].keys()) != set(chroms):
        for chrom in chroms:
            if chrom not in DNA_features[feature]:
                DNA_features[feature][chrom] = SparseVector(len(DNA[chrom]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1054/1054 [00:00<00:00, 141041.23it/s]


In [10]:
mode = 'hg19'
ASSEMBLY_d[mode] = ASSEMBLY
chroms_d[mode] = chroms
all_features_d[mode] = all_features
groups_d[mode] = groups
feature_names_d[mode] = feature_names
ZDNA_d[mode] = ZDNA
black_list_d[mode] = black_list
DNA_d[mode] = DNA
DNA_features_d[mode] = DNA_features

In [11]:
intersect = {i.upper() for i in DNA_features_d['mm9']} & {i.upper() for i in DNA_features_d['hg19']}

In [12]:
def to_bed(source, name):
    buf = []
    for chrm in source[name]:
        beds = source[name][chrm].indices[source[name][chrm].data != 0]
        data = source[name][chrm].data[source[name][chrm].data != 0].astype(float).astype(int)
        ends = np.append(source[name][chrm].indices[1:], 
              [source[name][chrm].shape])[source[name][chrm].data != 0]

        buf.extend([[chrm, beds[i], ends[i], data[i]] for i in range(len(beds))])
    buf = np.array(buf)
    df = pd.DataFrame([buf[:, 0], buf[:, 1], buf[:, 2], '', buf[:, 3], "", buf[:, 1], buf[:, 2], ""]).T
    df[3] = ''
    df[5] = "+"
    df[8] = ''
    return df

In [13]:
#gen = 'hg19'
#
#for name in tqdm([i for i in DNA_features_d[gen] if i.upper() in intersect and 'TFs and others' in i]):
#    df = to_bed(DNA_features_d[gen], name)
#    df.to_csv(f'tmp/{gen}_{name}.bed', sep = '\t', index=False, header=None)
##     break

In [14]:
def func(DNA_features_d, name, gen):
    df = to_bed(DNA_features_d, name)
    df.to_csv(f'tmp/{gen}_{name}.bed', sep = '\t', index=False, header=None)

In [15]:
#from joblib import Parallel, delayed
#gen = 'hg19'
#Parallel(n_jobs = -1)(delayed(func)(DNA_features_d[gen], name, gen)
#                     for name in tqdm([i for i in DNA_features_d[gen] 
#                                       if i.upper() in intersect and 'TFs and others' in i]))

# Data part

In [15]:
mode = 'mm9'
ASSEMBLY = ASSEMBLY_d[mode]
chroms = chroms_d[mode]
all_features = all_features_d[mode]
groups = groups_d[mode]
feature_names = feature_names_d[mode]
ZDNA = ZDNA_d[mode]
black_list = black_list_d[mode]
DNA = DNA_d[mode]
DNA_features = DNA_features_d[mode]

In [16]:
width = 512

np.random.seed(10)

ints_in = []
ints_out = []


for chrm in chroms:
    for st in trange(0, ZDNA[chrm].shape[0] - width, width):
        interval = [st, min(st + width, ZDNA[chrm].shape[0])]
        N_count = sum([bp == "N" for bp in DNA[chrm][interval[0]:interval[1]]])
        bl_count = black_list[chrm][interval[0]:interval[1]].sum()
        if N_count > width / 2 or bl_count > 0:
            continue
        else:
            if ZDNA[chrm][interval[0]: interval[1]].any():
                ints_in.append([chrm, int(interval[0]), int(interval[1]), 1])
            else:
                ints_out.append([chrm, int(interval[0]), int(interval[1]), 0])


                
                
print(len(ints_in))
print(len(ints_out))

ints_in_full = ints_in
ints_out_full = ints_out
#29745
#5443119

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 31059/31059 [00:01<00:00, 25283.35it/s]

17017
4843301


In [17]:
ints_in = ints_in_full
ints_out = [ints_out_full[i] for i in np.random.choice(range(len(ints_out_full)), 
                                                    size=len(ints_in) * 2, replace=False)]
# ints_out = ints_out_full

print(len(ints_in))
print(len(ints_out))
#484 for len 1000 
#9680 for len 1000

#629 for len 512
#12580 for len 5121

17017
34034


In [18]:
equalized = ints_in + ints_out

In [19]:
divisions = list(StratifiedKFold(5, shuffle=True, 
                                 random_state=42).split(equalized, [f"{elem[3]}_{elem[0]}"
                                         for i, elem 
                                         in enumerate(equalized)]))

In [20]:
dump([equalized, divisions], 'mm_divisions_kouzine.pkl', 3)

['mm_divisions_kouzine.pkl']

In [16]:
class Dataset(data.Dataset):
    def __init__(self, chroms, features, 
                 dna_source, features_source, 
                 labels_source, intervals, tokenizer):

        self.chroms = chroms
        self.features = features
        self.dna_source = dna_source
        self.features_source = features_source
        self.labels_source = labels_source
        self.intervals = intervals
        self.le = LabelBinarizer().fit(np.array([["A"], ["C"], ["T"], ["G"]]))
        self.configs = {
                        'ZHUNT_AS': {
                                'CG': 0, 'GC': 1, 'CA': 0, 'AC': 1, 
                                'TG': 0, 'GT': 1, 'TA': 1, 'AT': 1, 
                                'CC': 0, 'GG': 0, 'CT': 1, 'TC': 1, 
                                'GA': 1, 'AG': 1, 'AA': 1, 'TT': 1},
                       }
        seqs = (["A", "C", "T", "G"] + 
                ['AC', 'AT', 'AG', 'CT', 'CG', 'GT'] +
                ['AAC', 'ACC', 'AAT', 'ATT', 'AAG', 'AGG', 
                 'CCA', 'CAA', 'CCT', 'CTT', 'CCG', 'CGG', 
                 'TTA', 'TAA', 'TTC', 'TCC', 'TTG', 'TGG', 
                 'GGA', 'GAA', 'GGC', 'GCC', 'GGT', 'GTT'] +
                ['AAAC', 'AAAT', 'AAAG', 'CCCA', 'CCCT', 'CCCG',
                 'TTTA', 'TTTC', 'TTTG', 'GGGA', 'GGGC', 'GGGT'])
        self.tars = np.array([self.le.transform(list(i * 11)[:11]) for i in seqs])[:, ::-1, ::-1]
        # purine-pyrimidine
        self.tars = np.concatenate((self.tars, np.array([self.tars[4] + self.tars[9]])))
        self.tokenizer = tokenizer
        
        
    def __len__(self):
        return len(self.intervals)
    
    def __getitem__(self, index):
        interval = self.intervals[index]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        ll = list(self.dna_source[chrom][begin:end].upper())
        y = self.labels_source[interval[0]][interval[1]: interval[2]]        
        
        
#         DNA PART
        
        dna_OHE = self.le.transform(ll)[None]
        
        res = pd.DataFrame(convolve(dna_OHE, self.tars)[:, 5:-5, 3].T / 11)
        res = (res.rolling(5, min_periods=1).max().values == 1).astype(int)
        
        
#         ZHUNT PART
        zhunts = []
        for key in self.configs:
            vec = np.array(ll)
            vec = np.vectorize(lambda x:self.configs[key].get(x, 0))(
                                    np.char.add(vec[1:], vec[:-1]))
            zhunts.append(np.concatenate([vec, [0]]))
        
        
        # FEATURES PART
        feature_matr = []
        for feature in self.features:
            source = self.features_source[feature]
            feature_matr.append(source[chrom][begin:end])
        
        # UNION
        if len(feature_matr) > 0:
            X = np.hstack((
                           res,
                           np.array(zhunts).T, 
                           np.array(feature_matr).T/1000)).astype(np.float32)
#             X = (np.array(feature_matr).T/1000).astype(np.float32)
        else:
            X = dna_OHE.astype(np.float32)
        
        #K-mer part
        
        k_mers = seq2kmer(self.dna_source[chrom][begin:end+5].upper(),6)
        encoded_k_mers = self.tokenizer.encode_plus(k_mers, add_special_tokens=False, max_length=512)["input_ids"]

        return torch.Tensor(X), torch.Tensor(y).long(), ll, torch.LongTensor(encoded_k_mers), (chrom, begin, end)


In [17]:
import torch
from transformers import BertModel, BertConfig, PreTrainedTokenizer, BasicTokenizer, BertForTokenClassification
import collections
from torch.utils.data import DataLoader
import sklearn
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss

In [18]:
from dna_tokenizer import DNATokenizer, seq2kmer

In [19]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
for MODEL_NUMBER in range(5):

    train_inds, test_inds = divisions[MODEL_NUMBER]
    train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]

    random.shuffle(train_intervals)
    random.shuffle(test_intervals)
    
    train_dataset = Dataset(chroms, 
                        [i for i in feature_names if i.upper() in intersect], 
                       DNA, DNA_features, ZDNA, train_intervals, 
                        DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False))

    test_dataset = Dataset(chroms, 
                       [i for i in feature_names if i.upper() in intersect], 
                       DNA, DNA_features, ZDNA, test_intervals,
                          DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False))
    
    dump((train_dataset, test_dataset), f'ds_w_seq_mm_fold{MODEL_NUMBER}_kouzine.pkl')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected 

In [20]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        features, labels, sequences, input_ids, intervals = batch            
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        #print(model.device, input_ids.device, labels.device)
        outputs = model(input_ids = input_ids, labels = labels)        
        #print(outputs)
        loss, tr_logits = outputs['loss'], outputs['logits']
        #print(model(input_ids=ids, attention_mask=mask, labels=labels))
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 1000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 1000 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=0.1
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [21]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, eval_scores = [], [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            features, labels, sequences, input_ids, intervals = batch            
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            outputs = model(input_ids = input_ids, labels = labels)
            loss, eval_logits = outputs['loss'], outputs['logits']            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            flattened_scores = active_logits[:,1] - active_logits[:,0]
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            eval_scores.extend(flattened_scores)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [id.item() for id in eval_labels]
    predictions = [id.item() for id in eval_preds]
    scores = [id.item() for id in eval_scores]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, scores

In [22]:
device = 1
lr = 1e-5
EPOCHS = 3

dir_to_pretrained_model = "6-new-12w-0/"
config = BertConfig.from_pretrained('https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-6/config.json')
tokenizer = DNATokenizer.from_pretrained('6-new-12w-0/')

for MODEL_NUMBER in range(5):
    train_dataset, test_dataset = load(f'ds_w_seq_mm_fold{MODEL_NUMBER}_kouzine.pkl')
    training_loader = DataLoader(train_dataset, batch_size=24, num_workers = 2)
    testing_loader = DataLoader(test_dataset, batch_size=16, num_workers = 2)


    model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model, config=config)
    model.to(device)
    
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(training_loader), epochs=EPOCHS)

    for epoch in range(EPOCHS):
        print(f"Training fold {MODEL_NUMBER} epoch: {epoch + 1}")
        train(epoch)
        labels, predictions, scores = valid(model, testing_loader)
        print(f'Fold {MODEL_NUMBER} validation ROC-AUC: ', roc_auc_score(labels, scores))

    print(sklearn.metrics.classification_report(labels, predictions))
    model.save_pretrained(f'dnabert_mm_fold_{MODEL_NUMBER}_kouzine')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
Some weights of the model checkpoint at 6-new-12w-0/ were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint

Training fold 0 epoch: 1


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training loss per 1000 training steps: 0.7216289639472961
Training loss per 1000 training steps: 0.2036324538054181
Training loss epoch: 0.13481990928590937
Training accuracy epoch: 0.9372454119158233


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Validation loss per 100 evaluation steps: 0.04173064976930618
Validation loss per 100 evaluation steps: 0.024915910842321295
Validation loss per 100 evaluation steps: 0.02741922919212178
Validation loss per 100 evaluation steps: 0.027873950557894635
Validation loss per 100 evaluation steps: 0.027659079785159142
Validation loss per 100 evaluation steps: 0.027502683468140707
Validation loss per 100 evaluation steps: 0.02765760065665385
Validation Loss: 0.027903418464029387
Validation Accuracy: 0.986402819040982
Fold 4 validation ROC-AUC:  0.9946813208252935
Training fold 4 epoch: 3


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training loss per 1000 training steps: 0.03428428992629051
Training loss per 1000 training steps: 0.026661128611662067
Training loss epoch: 0.026647961104084048
Training accuracy epoch: 0.9871686722413162


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Validation loss per 100 evaluation steps: 0.04064717888832092
Validation loss per 100 evaluation steps: 0.024351452265192967
Validation loss per 100 evaluation steps: 0.026515456378710136
Validation loss per 100 evaluation steps: 0.02712964755328267
Validation loss per 100 evaluation steps: 0.026808861492694055
Validation loss per 100 evaluation steps: 0.026675443761719916
Validation loss per 100 evaluation steps: 0.02684886656320358
Validation Loss: 0.027058914514383848
Validation Accuracy: 0.9869845156005478
Fold 4 validation ROC-AUC:  0.994852651622968
              precision    recall  f1-score   support

           0       0.99      0.99      0.99   5107137
           1       0.71      0.74      0.72    120383

    accuracy                           0.99   5227520
   macro avg       0.85      0.87      0.86   5227520
weighted avg       0.99      0.99      0.99   5227520

